## Notebook 1: A first Earth Engine workflow (NDVI over Missouri)

This notebook demonstrates a **minimal but complete workflow** using Google Earth Engine:

- connect to Earth Engine
- load a public satellite dataset
- compute a simple derived product (NDVI)
- visualize the result on a map

The goal is not to be clever or exhaustive. The goal is to see the full pattern once, end to end.

In [13]:
import ee
from google.cloud import storage

# 1. Set Workspace Identifiers
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"

# 2. Initialize Earth Engine (Required for GEE data access)
ee.Initialize(project=PROJECT)

# 3. Setup Cloud Storage Client (Required to read/write files to our bucket)
storage_client = storage.Client(project=PROJECT)

print(f"Project '{PROJECT}' initialized and storage client ready.")

Project 'eeps-geospatial' initialized and storage client ready.


## A note on Python packages

This notebook uses the `geemap` package to display interactive maps.

`geemap` is **not installed by default** in many cloud notebook environments. Because the terminal is disabled, we install it directly from the notebook using `%pip`.

This is normal in cloud workflows and does not mean anything is “broken.”

## Installing geemap (one-time step)

The next cell installs `geemap` into the current notebook environment.

- If this is your **first time running this notebook**, run the cell and then **restart the kernel**.
- After restarting, re-run the notebook from the top.
- If `geemap` is already installed, this step will be fast or skipped.

If there are packages you need to install, you can do it this way

In [14]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [15]:
import geemap.foliumap as geemap  # Use folium backend (more reliable in cloud environments)

## If something goes wrong here

If you see an error like:  
``ModuleNotFoundError: No module named ‘geemap’``  
then the kernel has not been restarted yet. Restart the kernel and run the notebook again from the top.

If errors persist, stop and ask for help — do not try random fixes.

## Define the area of interest

We'll use Missouri's boundary from the US Census TIGER dataset.

In [16]:
# Missouri boundary (public TIGER dataset)
states = ee.FeatureCollection("TIGER/2018/States")
mo = states.filter(ee.Filter.eq("NAME", "Missouri")).geometry()

## Compute NDVI from Sentinel-2 imagery

[NDVI (Normalized Difference Vegetation Index)](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) measures vegetation health using the difference between near-infrared and red light reflectance. Values range from -1 to 1, where higher values indicate healthier vegetation.

Here we:
1. Load Sentinel-2 surface reflectance imagery
2. Apply a cloud mask
3. Calculate NDVI for each image
4. Create a median composite for summer 2025

In [17]:
# Sentinel-2 SR harmonized
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Simple cloud mask using SCL (Scene Classification Layer)
def mask_s2_scl(img):
    scl = img.select("SCL")
    # Keep: vegetation, bare, water, etc. Mask: cloud/shadow/snow
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return img.updateMask(mask)

start = "2025-06-01"
end   = "2025-09-01"

s2_mo = (
    s2.filterBounds(mo)
      .filterDate(start, end)
      .map(mask_s2_scl)
)

# NDVI per image, then composite (median)
def add_ndvi(img):
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

ndvi_comp = s2_mo.map(add_ndvi).select("NDVI").median().clip(mo)

## Visualize the result

Display the NDVI composite on an interactive map. Green areas indicate healthy vegetation; brown/red areas indicate bare soil or water.

In [18]:
# Map
m = geemap.Map(center=[38.5, -92.5], zoom=6)

# NDVI color palette: brown (bare) -> yellow -> green (healthy vegetation)
ndvi_palette = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']

vis = {"min": 0.0, "max": 0.8, "palette": ndvi_palette}
m.addLayer(ndvi_comp, vis, f"NDVI median {start} to {end}")
m.addLayer(mo, {}, "Missouri boundary")
m

## Try it yourself

Ideas to explore:

1. **Change the date range**: What does winter NDVI look like? Try December-February
2. **Different state**: Replace "Missouri" with "Kansas" or "Iowa" - how do they compare?
3. **Zoom in**: Pick a county or draw a smaller region - can you see individual farm fields?
4. **Different index**: Try EVI instead of NDVI using bands B8, B4, and B2: `img.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {'NIR': img.select('B8'), 'RED': img.select('B4'), 'BLUE': img.select('B2')})`
5. **Export the result**: Save your NDVI image to Cloud Storage using `ee.batch.Export.image.toCloudStorage()`